In [1]:
import pandas as pd
from pulp import *

In [4]:
inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')


In [20]:
efficiency_scores = {}
reference_units = {}

for DMUo in inputs.index:
    problem = LpProblem(f"Efficiency_{DMUo}", LpMinimize)

    lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
    theta = LpVariable("theta", lowBound=0)  # Dodatkowa zmienna reprezentująca efektywność

    # Funkcja celu: minimalizacja theta
    problem += theta

    # Ograniczenia na nakłady
    for input_metric in inputs.columns:
        problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric]

    # Ograniczenia na produkcję
    for output_metric in outputs.columns:
        problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] for i in outputs.index]) >= outputs.loc[DMUo, output_metric]
    problem.solve(PULP_CBC_CMD(msg=False))

    # Wypisanie wyników
    if LpStatus[problem.status] == 'Optimal':
        efficiency_score = 1 / value(theta)  # Odwrócenie wyniku, ponieważ minimalizujemy theta
        print(f"Efficiency for {DMUo}: {efficiency_score}")
    else:
        print(f"Problem for {DMUo} is not solvable")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/joanna/miniconda3/envs/iswd/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/05/htkmyw9s0mj9r1tdvqjv8cg00000gn/T/8caa7611b48c48dba2d26b9bfbbe1f78-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/05/htkmyw9s0mj9r1tdvqjv8cg00000gn/T/8caa7611b48c48dba2d26b9bfbbe1f78-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 83 RHS
At line 90 BOUNDS
At line 91 ENDATA
Problem MODEL has 6 rows, 12 columns and 70 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 12 (0) columns and 70 (0) elements
Perturbing problem by 0.001% of 0.58465567 - largest nonzero change 1.2851015e-05 ( 0.0021980484%) - largest zero change 1.1190008e-05
0  Obj 0 Primal inf 90.892342 (2)
6  Obj 1.0001105
Optimal - objective value 1
Optimal objective 1 - 6 iterations 

In [26]:
from pulp import *

# Załaduj dane
inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')

# Przechowuje wyniki
efficiency_scores = {}
reference_units = {}

# Pętla dla każdego lotniska
for DMUo in inputs.index:
    problem = LpProblem(f"Efficiency_{DMUo}", LpMinimize)

    lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
    theta = LpVariable("theta", lowBound=0, upBound=1)  # Zmienna reprezentująca efektywność

    # Funkcja celu: minimalizacja theta
    problem += theta

    # Ograniczenia na nakłady
    for input_metric in inputs.columns:
        problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric]

    # Ograniczenia na produkcję
    for output_metric in outputs.columns:
        problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] for i in outputs.index]) >= outputs.loc[DMUo, output_metric]

    # Rozwiązanie problemu
    problem.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[problem.status] == 'Optimal':
        efficiency_score = value(theta)  # Theta jest już efektywnością
        efficiency_scores[DMUo] = efficiency_score

        # Zbieranie informacji o jednostkach referencyjnych dla nieefektywnych DMU
        if efficiency_score < 1:
            reference_units[DMUo] = {}
            for i in inputs.index:
                if lambdas[i].varValue > 0:
                    reference_units[DMUo][i] = lambdas[i].varValue

    else:
        print(f"Problem for {DMUo} is not solvable")

# Wydrukuj wyniki
for airport, efficiency in efficiency_scores.items():
    print(f"Efficiency for {airport}: {efficiency:.2f}")
    if airport in reference_units:
        print(f"Reference units for {airport}:")
        for ref, value in reference_units[airport].items():
            print(f"  {ref}: {value:.2f}")


Efficiency for WAW: 1.00
Efficiency for KRK: 1.00
Efficiency for KAT: 0.59
Reference units for KAT:
  KRK: 0.00
  WRO: 1.23
  GDN: 0.27
Efficiency for WRO: 1.00
Efficiency for POZ: 0.80
Reference units for POZ:
  WAW: 0.05
  KRK: 0.07
  WRO: 0.20
  GDN: 0.16
Efficiency for LCJ: 0.30
Reference units for LCJ:
  GDN: 0.16
  BZG: 0.06
Efficiency for GDN: 1.00
Efficiency for SZZ: 0.27
Reference units for SZZ:
  WAW: 0.00
  GDN: 0.15
  BZG: 0.06
Efficiency for BZG: 1.00
Efficiency for RZE: 0.41
Reference units for RZE:
  WRO: 0.12
  GDN: 0.05
  BZG: 0.03
Efficiency for IEG: 0.26
Reference units for IEG:
  GDN: 0.03
